In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from data_loader import load_data_for

In [34]:
# Import All Model Data
lr_model_results = load_data_for('logistic_regression')
rf_model_results = load_data_for('random_forest_classifier')
svc_model_results = load_data_for('support_vector_classifier')
xgbc_model_results = load_data_for('xgb_classifier')

# NOTE: Each dictionary contains the following keys: 
# 'best_estimators', 'best_scores', 'baseline_scores', 'X_test_unpreprocessed', 'X_test_preprocessed', 'y_test', 'predicted_labels'

In [36]:
print(lr_model_results['best_scores'])
print(rf_model_results['best_scores'])
print(svc_model_results['best_scores'])
print(xgbc_model_results['best_scores'])

[0.9771561225883225, 0.9771909114855175, 0.9767190196250475, 0.976736246586032, 0.9768539742301727]
[0.992907098844927, 0.9927753111868707, 0.993092307387366, 0.9931055660667056, 0.993778975314023]
[0.9938837920489296, 0.9939848433882744, 0.9940203348216495, 0.9938915124694329, 0.9934053529522433]
[0.9973943110776455, 0.9971861407524101, 0.9973542614227533, 0.9970967965676804, 0.9971366009769244]
